In [11]:
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime, timedelta
import folium
import pandas as pd
import seaborn as sns
import matplotlib.pyplot 
import numpy 



In [12]:
data = pd.read_csv('Chiraq.csv') #Data de tout les cambriolages à Chicago de 2018
data = data.dropna() #Suppression des lignes sans valeurs
sns.set_style('darkgrid')
df=pd.DataFrame(data=data) #conversion csv en tableau de données
df

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
4,11231100,JB152555,02/15/2018 01:00:00 AM,067XX N NEWGARD AVE,650,BURGLARY,HOME INVASION,APARTMENT,False,False,...,49,1,5,1165377.0,1944782.0,2018,03/25/2020 03:45:43 PM,42.004081,-87.666926,"(42.004080979, -87.66692551)"
5,11514213,JB525083,11/22/2018 04:00:00 AM,065XX W IRVING PARK RD,610,BURGLARY,FORCIBLE ENTRY,OTHER (SPECIFY),False,False,...,38,17,5,1132190.0,1925884.0,2018,08/16/2020 03:47:58 PM,41.952867,-87.789464,"(41.952866977, -87.789463625)"
13,11228336,JB148467,02/11/2018 04:30:00 PM,020XX W ADAMS ST,610,BURGLARY,FORCIBLE ENTRY,SCHOOL - PUBLIC BUILDING,True,False,...,27,28,5,1162662.0,1899079.0,2018,03/27/2020 03:47:00 PM,41.878727,-87.678199,"(41.878726928, -87.678199026)"
14,11407559,JB384794,08/08/2018 02:09:00 PM,015XX E 86TH ST,620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE-GARAGE,True,False,...,8,45,5,1188016.0,1848264.0,2018,06/09/2019 03:57:16 PM,41.738718,-87.586726,"(41.738718396, -87.586726409)"
15,11250202,JB177917,03/08/2018 08:55:00 AM,053XX W MELROSE ST,610,BURGLARY,FORCIBLE ENTRY,RESIDENCE - GARAGE,False,False,...,30,15,5,1140152.0,1921089.0,2018,03/29/2020 03:43:48 PM,41.939567,-87.760312,"(41.939566652, -87.760312257)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11742,11401393,JB377137,07/30/2018 12:01:00 AM,043XX N KEDVALE AVE,630,BURGLARY,ATTEMPT FORCIBLE ENTRY,APARTMENT,False,False,...,39,16,5,1148069.0,1928545.0,2018,09/15/2021 04:50:41 PM,41.959878,-87.731022,"(41.959877611, -87.731022096)"
11743,11461211,JB455165,09/28/2018 01:00:00 PM,013XX W ARGYLE ST,620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,False,False,...,47,3,5,1166432.0,1933324.0,2018,09/16/2021 04:44:39 PM,41.972617,-87.663374,"(41.972617333, -87.663373937)"
11744,11452376,JB443460,09/19/2018 11:29:00 PM,005XX S STATE ST,620,BURGLARY,UNLAWFUL ENTRY,RESTAURANT,False,False,...,4,32,5,1176446.0,1897944.0,2018,09/29/2021 04:51:01 PM,41.875313,-87.627622,"(41.875312543, -87.627621596)"
11745,11250846,JB178739,03/08/2018 08:15:00 PM,017XX S DESPLAINES ST,610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,...,11,31,5,1172335.0,1891796.0,2018,10/06/2021 04:49:11 PM,41.858534,-87.642897,"(41.858533746, -87.642896985)"


In [13]:
# Time Conversion Function
def time_convert(date_time):
    s1 = date_time[:11]
    s2 = date_time[11:]
    
    month = s1[:2]
    date = s1[3:5]
    year = s1[6:10]
    
    hr = s2[:2]
    mins = s2[3:5]
    sec = s2[6:8]
    time_frame = s2[9:]
    if(time_frame == 'PM'):
        if (int(hr) != 12):
            hr = str(int(hr) + 12)
    else:
        if(int(hr) == 12):
            hr = '00'
    
    final_date = datetime(int(year), int(month), int(date), int(hr), int(mins), int(sec))
    return final_date

In [14]:
# Using apply() of pandas to apply time_convert on every row of the Date column
df['Date'] = df['Date'].apply(time_convert)
df.head() #Affichage des premières lignes

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
4,11231100,JB152555,2018-02-15 01:00:00,067XX N NEWGARD AVE,650,BURGLARY,HOME INVASION,APARTMENT,False,False,...,49,1,5,1165377.0,1944782.0,2018,03/25/2020 03:45:43 PM,42.004081,-87.666926,"(42.004080979, -87.66692551)"
5,11514213,JB525083,2018-11-22 04:00:00,065XX W IRVING PARK RD,610,BURGLARY,FORCIBLE ENTRY,OTHER (SPECIFY),False,False,...,38,17,5,1132190.0,1925884.0,2018,08/16/2020 03:47:58 PM,41.952867,-87.789464,"(41.952866977, -87.789463625)"
13,11228336,JB148467,2018-02-11 16:30:00,020XX W ADAMS ST,610,BURGLARY,FORCIBLE ENTRY,SCHOOL - PUBLIC BUILDING,True,False,...,27,28,5,1162662.0,1899079.0,2018,03/27/2020 03:47:00 PM,41.878727,-87.678199,"(41.878726928, -87.678199026)"
14,11407559,JB384794,2018-08-08 14:09:00,015XX E 86TH ST,620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE-GARAGE,True,False,...,8,45,5,1188016.0,1848264.0,2018,06/09/2019 03:57:16 PM,41.738718,-87.586726,"(41.738718396, -87.586726409)"
15,11250202,JB177917,2018-03-08 08:55:00,053XX W MELROSE ST,610,BURGLARY,FORCIBLE ENTRY,RESIDENCE - GARAGE,False,False,...,30,15,5,1140152.0,1921089.0,2018,03/29/2020 03:43:48 PM,41.939567,-87.760312,"(41.939566652, -87.760312257)"


In [15]:
## Crime data distribution of gang based criminal activities
import folium
chicago_map = folium.Map(location=[41.864073,-87.706819],zoom_start=10,tiles="openstreetmap")
color=['aliceblue','steelblue','teal','wheat','saddlebrown','lavender','khaki','darkslategrey','darkmagenta','darkgoldenrod','aliceblue','indigo','red','blue','green','black','white','yellow','brown','grey','orange','purple','antiquewhite','beige','azure','burlywood','burlywood','burlywood','burlywood','burlywood','burlywood','burlywood','burlywood','burlywood']
for i in range(5):
    date=pd.Timestamp('2018-01-01')+timedelta(days=i)
    for index,row in df.iterrows():
        if(row['Date'].strftime("%Y-%m-%d")==date.strftime("%Y-%m-%d")):
            folium.CircleMarker(location=[row['Latitude'],row['Longitude']], popup=row['Primary Type'],color=color[row['District']], fill = True).add_to(chicago_map)
    img = Image.open( Image.convert(chicago_map))
    d=ImageDraw.Draw(img)
    fnt  = ImageFont.truetype("comicbd.ttf",25)
    d.text((60,60),"jour 1",font=fnt, fill=(255,255,255))
    chicago_map.save(date.strftime("%Y-%m-%d")+'map.html')
    chicago_map = folium.Map(location=[41.864073,-87.706819],zoom_start=11,tiles="openstreetmap")


#chicago_map._build_map()
chicago_map

AttributeError: module 'PIL.Image' has no attribute 'convert'

In [ ]:
matplotlib.colors.cnames

In [22]:
import folium
chicago_map = folium.Map(location=[41.864073,-87.706819],zoom_start=10,tiles="CartoDB dark_matter")
color=['indigo','red','blue','green','hotpink','white','yellow','brown','grey','orange','purple','antiquewhite','beige','azure','burlywood','darkgoldenrod','cornsilk','crimson','cadetblue','blanchedalmond','greenyellow','bisque','hotpink','aliceblue','steelblue','teal','wheat','saddlebrown','lavender','khaki','darkslategrey','darkmagenta','darkgoldenrod','aliceblue']
for i in range(2):
    chicago_map = folium.Map(location=[41.864073,-87.706819],zoom_start=10,tiles="CartoDB dark_matter")
    date=pd.Timestamp('2018-01-01')+timedelta(days=i)
    for index,row in df.iterrows():
        if(row['Date'].strftime("%Y-%m-%d")==date.strftime("%Y-%m-%d")):
            folium.CircleMarker(location=[row['Latitude'],row['Longitude']], popup=row['Primary Type'],color=color[row['District']], fill = True).add_to(chicago_map)
    #chicago_map.save(date.strftime("%Y-%m-%d")+'map.html')


chicago_map
#chicago_map._build_map()
#chicago_map